Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

We will use the results of the estimation using W=Identity

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters


using DataFrames
using BenchmarkTools



In [2]:
using NBInclude
@nbinclude("solve_model_cf.ipynb")

#importou a função solve_tree

In [3]:
#Vh/D, L/D, Dj/D
data = [1.32, 0.32, 0.78]; 

#ρ, β, c0, λj
game_parameters = [0.52, 22.59, 0.2, 0.01];

#counterfactual parameters####
D = Float64(1.0)

#simulation of baseline model, tsym=1
tsym = Int64(1)
dip = Float64(0.00)

additional_cost = dip * D
ac_s = additional_cost
ac_j = additional_cost

counterfactual_parameters = [tsym, additional_cost, ac_s, ac_j];

In [4]:
s_W, j_W, Pst_array, Pjt_array, Ds, Dj = solve_tree(data, game_parameters, counterfactual_parameters);

  4.946115 seconds (44.49 M allocations: 921.648 MiB, 3.87% gc time)
  7.156414 seconds (43.22 M allocations: 858.002 MiB, 1.10% gc time)
  8.588299 seconds (43.22 M allocations: 858.002 MiB, 0.83% gc time)
  8.894079 seconds (40.16 M allocations: 811.310 MiB, 0.79% gc time)
 30.395748 seconds (171.13 M allocations: 3.370 GiB, 1.36% gc time)


Simulation parameters

In [22]:
#θs0, θj0, μ
simulation_parameters = [45, 99, 3.11];

### Criando funções para simular o jogo

In [23]:
θs0, θj0, μ = simulation_parameters;
θs0 = Int64(θs0);
θj0 = Int64(θj0);

#parameters to some functions that are similar as the ones in solve_model

c0 = game_parameters[3];
c1 = Float64(0.0015);

ρ = game_parameters[1];
λj = game_parameters[4];


Vmax = data[1];
L = data[2];
Dj = data[3];

D = Float64(1.0)
Ds = D - Dj;


In [24]:
#counterfactual parameters
tsym = counterfactual_parameters[1]
additional_cost = counterfactual_parameters[2]
ac_s = counterfactual_parameters[3]
ac_j = counterfactual_parameters[4];

In [26]:
#cost function
function Ct(t)
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
    end
end



#liquidation payoffs

function s_L(t, additional_cost=ac_s, Ds=Ds)
    S_L = min(L - Ct(t) - additional_cost, Ds)
    return max(S_L, zero(0.0))
end

function j_L(t, additional_cost=ac_j, Dj=Dj)
    J_L = min(L - Ct(t) - additional_cost - s_L(t), Dj)
    return max(J_L, zero(0.0))
end


function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t, additional_cost)

    if(t <=1)
        return Vmax - additional_cost
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax - additional_cost
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, additional_cost, ac_s, ac_j)
    
    liq = (m_L(t), Vt(Vmax, ρ, t, additional_cost) * hm_next/100 - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next, t, tsym)
    
    if(t<=tsym)
        
        #t <= tsym, there is symmetric information
        
         if(propositor=="s")
            hst = hk_next
            lst = hk_next

            hjt = hm_next
            ljt = hm_next

        else
            hst = hm_next
            lst = hm_next

            hjt = hk_next
            ljt = hk_next
        end
        
    else
        
        #t > tsym, there is asymmetric information


        if(propositor=="s")
            hst = hk_next
            lst = lk_next

            hjt = hm_next
            ljt = lm_next

        else
            hst = hm_next
            lst = lm_next

            hjt = hk_next
            ljt = lk_next
        end
    end

    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [27]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == 100)
        
        return 100

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [28]:
function choose_parameters(propositor, hst, lst, hjt, ljt)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




#test
Hs0 = 30
Hj0 = 28


hst = Hs0
hjt = Hj0

#assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
lst = hst
ljt = hjt




choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [29]:
function simulate_game(Hs0, Hj0, μ, λj, counterfactual_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    hst = Hs0
    hjt = Hj0

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    #counterfactual parameters
    tsym = counterfactual_parameters[1]
    additional_cost = counterfactual_parameters[2]
    ac_s = counterfactual_parameters[3]
    ac_j = counterfactual_parameters[4];
    
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt)
        
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next, t, tsym)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next, t, tsym)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, additional_cost, ac_s, ac_j)

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0) #to avoid observed_proposals==0.0
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [30]:
# simulate_game(θs0, θj0, μ, λj, s_W, j_W, Pst_array, Pjt_array)

In [31]:
function loop_simulations(S, N, θs0, θj0, μ, λj, counterfactual_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    #S is the number of simulations
    #N is the number of observations
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    Results[:,:, end-1] .= Dj
    Results[:,:, end] .= Ds


    for s in 1:S
        for n in 1:N
            Results[s,n,1:5] .= simulate_game(θs0, θj0, μ, λj, counterfactual_parameters)
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [33]:
function simulate_moments(S, N, θs0, θj0, μ, λj, counterfactual_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    
    Results = loop_simulations(S, N, θs0, θj0, μ, λj, counterfactual_parameters)
    
    
    Moments = zeros(S, 8)
    
    
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Dj = Results[s,:,6];
            DF.Ds = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)

            #if there are no precourt cases, it will return zero
            if(isnan(mm5))
                mm5 = zero(0.0)
            end


            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)

            #if there are no precourt cases, it will return zero
            if(isnan(mm6))
                mm6 = zero(0.0)
            end


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)



            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end

simulate_moments (generic function with 5 methods)

In [40]:
#test
S = 40
N = 66


simulated_moments = simulate_moments(S, N, θs0, θj0, μ, λj, counterfactual_parameters)

8-element Array{Float64,1}:
 0.792733250976022
 1.0
 2.049460821895136
 0.9920454545454545
 0.480598918918919
 0.41946597933071417
 0.7361956293770641
 0.49562729616635703

Comparing with the actual data moments

In [38]:
#data_moments
data_moments = [0.728, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497];

In [43]:
function error_vector(data_moments, simulated_moments)
    #two arrays with the data moments and the simulated moments
    
# #     return (simulated_moments .- data_moments) ./ data_moments
#     
    return (simulated_moments .- data_moments)
    
end

error_vector (generic function with 1 method)

In [44]:
error_vector(data_moments, simulated_moments)

8-element Array{Float64,1}:
  0.064733250976022
  0.15700000000000003
 -0.710539178104864
  0.02304545454545448
 -0.14290108108108107
 -0.13743402066928578
 -0.036804370622935956
 -0.0013727038336429631

Derivative of moments w.r.t. parameters

In [46]:
game_parameters
simulation_parameters

3-element Array{Float64,1}:
 45.0
 99.0
  3.11

In [48]:
0.52 * 101/100

0.5252